In [1]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2


In [2]:
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, TrainerCallback
import os



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Load the Dataset**

In [4]:
dataset = load_dataset('paws','labeled_final')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

labeled_final/train-00000-of-00001.parqu(…):   0%|          | 0.00/8.43M [00:00<?, ?B/s]

labeled_final/test-00000-of-00001.parque(…):   0%|          | 0.00/1.24M [00:00<?, ?B/s]

labeled_final/validation-00000-of-00001.(…):   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/49401 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8000 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 49401
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 8000
    })
})

In [6]:
dataset['train'][0]

{'id': 1,
 'sentence1': 'In Paris , in October 1560 , he secretly met the English ambassador , Nicolas Throckmorton , asking him for a passport to return to England through Scotland .',
 'sentence2': 'In October 1560 , he secretly met with the English ambassador , Nicolas Throckmorton , in Paris , and asked him for a passport to return to Scotland through England .',
 'label': 0}

In [8]:
df= pd.DataFrame(dataset['train'])

In [9]:
df

,id,sentence1,sentence2,label
0,1,"In Paris , in October 1560 , he secretly met t...","In October 1560 , he secretly met with the Eng...",0
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
2,3,"There are also specific discussions , public p...","There are also public discussions , profile sp...",0
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
...,...,...,...,...
49396,49397,"`` Our school is of spiritual and spiritual , ...",`` Our School is of the Temporal and the Spiri...,0
49397,49398,She was in Cork on June 24 and arrived on 8 Ju...,"She was at Cork on 24 June , and arrived in th...",1
49398,49399,Cornelia Stuyvesant Vanderbilt ( George and Ed...,John John F. A. Cecil ( the only child of Geor...,0
49399,49400,The third season was premiered on 7 June 2010 ...,"The fourth season was premiered on June 7 , 20...",0


In [10]:
len(df)

49401

In [11]:
df.duplicated().sum()

np.int64(0)

In [12]:
df.nunique()

,0
id,49401
sentence1,30675
sentence2,30724
label,2


In [13]:
df = df[df["label"]==1]
df

,id,sentence1,sentence2,label
1,2,The NBA season of 1975 -- 76 was the 30th seas...,The 1975 -- 76 season of the National Basketba...,1
3,4,When comparable rates of flow can be maintaine...,The results are high when comparable flow rate...,1
4,5,It is the seat of Zerendi District in Akmola R...,It is the seat of the district of Zerendi in A...,1
5,6,William Henry Henry Harman was born on 17 Febr...,"William Henry Harman was born in Waynesboro , ...",1
7,8,With a discrete amount of probabilities Formul...,Given a discrete set of probabilities formula ...,1
...,...,...,...,...
49384,49385,"The Romanesque language , Galician ( Galego ) ...",The Romance language currently spoken in Galic...,1
49390,49391,Note that k is a vector consisting of three in...,It is necessary to note that k is a vector con...,1
49393,49394,"Tim Henman won in the final 6 -- 2 , 7 -- 6 , ...","Tim Tim Henman won 6 -- 2 , 7 -- 6 against Yev...",1
49395,49396,He was considered an active member of the coun...,He was considered an active member of the Coun...,1


In [15]:
def preprocess_pows(dataset, label=1):
    df = pd.DataFrame(dataset)     # FIXED
    df = df[df["label"] == label]

    df['input'] = "Paraphrase: " + df['sentence1']
    df['target_text'] = df['sentence2']   # FIXED name

    return df[['input', 'target_text']]


train_df = preprocess_pows(dataset['train']).sample(7000, random_state=42)
test_df = preprocess_pows(dataset['test']).sample(300, random_state=42)
validation_df = preprocess_pows(dataset['validation']).sample(300, random_state=42)


In [16]:
train_df

,input,target_text
28667,Paraphrase: William died in 1859 and Elizabeth...,"In 1859 , William and Elizabeth died the follo..."
33659,Paraphrase: The leaves are generally 1.5-4 mm ...,The leaves are typically 1.5-4 mm wide and 0.2...
17421,Paraphrase: Drummer Wayne Bennett left the ban...,"In June 2007 , drummer Wayne Bennett left the ..."
4012,Paraphrase: Their skin also secretes chemicals...,"Also , their skin secretes chemicals that are ..."
46074,Paraphrase: When the Florentine Republic fell ...,"When the Florentine Republic fell in 1530 , Vo..."
...,...,...
3724,"Paraphrase: Hislop married his wife , Desha , ...","Hislop married his wife Desha in 1995 , and is..."
10500,Paraphrase: Some windows on the first floor at...,Some first floor windows at the western end of...
48821,Paraphrase: The song was composed by Unnikumar...,"This song was composed by Unnikumar , sung by ..."
43904,"Paraphrase: On March 16 , 1494 , Maximilian Bi...",Bianca Maria Sforza was married to Maximilian ...


In [17]:
type(train_df)

pandas.core.frame.DataFrame

In [18]:
train_df = Dataset.from_pandas(train_df)
test_df = Dataset.from_pandas(test_df)
validation_df= Dataset.from_pandas(validation_df)

In [19]:
train_df[0]

{'input': 'Paraphrase: William died in 1859 and Elizabeth died the following year .',
 'target_text': 'In 1859 , William and Elizabeth died the following year .',
 '__index_level_0__': 28667}

## ***Tokenization***

In [20]:
#initialize tokenizer and model
model_name = 't5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

{'input': 'Paraphrase: William died in 1859 and Elizabeth died the following year .',
 'target_text': 'In 1859 , William and Elizabeth died the following year .',
 '__index_level_0__': 28667,
 'input_ids': [4734,
  27111,
  10,
  4599,
  3977,
  16,
  507,
  3390,
  11,
  9066,
  3977,
  8,
  826,
  215,
  3,
  5,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


In [21]:
max_length = max([len(tokenizer.encode(text)) for text in train_df['input']])
max_length


def tokenize_function(examples):
    inputs = tokenizer(examples['input'], padding='max_length', truncation=True, max_length=max_length)
    traget= tokenizer(examples['target_text'], padding='max_length', truncation=True, max_length=max_length)

    inputs['labels'] = traget['input_ids']
    return inputs

#tokenizer datasets
train_df = train_df.map(tokenize_function, batched=True)

validation_df = validation_df.map(tokenize_function, batched=True)

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [23]:
train_df.column_names

['input',
 'target_text',
 '__index_level_0__',
 'input_ids',
 'attention_mask',
 'labels']

In [24]:
train_df['input'][1]

'Paraphrase: The leaves are generally 1.5-4 mm wide and 0.2-0.7 mm long .'

In [25]:

train_df['target_text'][1]

'The leaves are typically 1.5-4 mm wide and 0.2-0.7 mm long .'

In [26]:
def show_row_details(dataset, idx):
    """
    Show all details of a single row in a HuggingFace Dataset.

    Args:
        dataset : HuggingFace Dataset
        idx     : row index (int)
    """
    row = dataset[idx]  # get row
    print(f"Row index: {idx}\n")


    for key, value in row.items():
      print(f"{key}: {value}\n")

# Example usage:
show_row_details(train_df, 1)



Row index: 1

input: Paraphrase: The leaves are generally 1.5-4 mm wide and 0.2-0.7 mm long .

target_text: The leaves are typically 1.5-4 mm wide and 0.2-0.7 mm long .

__index_level_0__: 33659

input_ids: [4734, 27111, 10, 37, 3231, 33, 2389, 8613, 4278, 3, 635, 1148, 11, 3, 18189, 18, 22426, 3, 635, 307, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

labels: [37, 3231, 33, 3115, 8613, 4278, 3, 635, 1148, 11, 3, 18189, 18, 22426, 3, 635, 307, 3, 5, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

## **Fine Tunning**

In [27]:
results_dir='/content/drive/MyDrive/results'
model_dir='/content/drive/MyDrive/saved_t5_models'


os.makedirs(results_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [28]:
# # Define training arguments
# training_args = TrainingArguments(
#     output_dir=results_dir,  # Directory where the results (logs, model outputs) will be saved
#     overwrite_output_dir=True,  # Overwrite the output directory if it exists
#     evaluation_strategy="epoch",  # Evaluate the model at the end of each epoch
#     eval_steps=500,  # Evaluate every 500 steps (this might not be necessary if evaluation is at epoch)
#     logging_dir='./logs',  # Directory to save logs
#     logging_steps=100,  # Log every 100 steps
#     learning_rate=3e-5,  # Learning rate for optimizer
#     per_device_train_batch_size=8,  # Batch size for training
#     per_device_eval_batch_size=8,  # Batch size for evaluation
#     num_train_epochs=5,  # Number of epochs to train the model
#     weight_decay=0.01,  # L2 regularization to prevent overfitting
#     save_strategy="no",  # Disable saving checkpoints during training to save memory
#     fp16=True,  # Use mixed precision training to speed up training and reduce memory usage
#     report_to="none"  # Disable reporting to avoid unwanted logging to external platforms
# )

In [29]:
training_args = TrainingArguments(
    output_dir=results_dir,
    overwrite_output_dir=True,
    logging_dir='./logs',
    logging_steps=100,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy="no",
    fp16=True,
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,  # The model to be trained
    args=training_args,  # The training arguments defined above
    train_dataset=train_df,  # The training dataset
    eval_dataset=validation_df,  # The validation dataset
)

# Start training
trainer.train()  # Begin the training process

Step,Training Loss
100,2.355900
200,0.287300
300,0.245700
400,0.225700
500,0.220900
600,0.230200
700,0.223700


Step,Training Loss
100,2.355900
200,0.287300
300,0.245700
400,0.225700
500,0.220900
600,0.230200
700,0.223700
800,0.209800
900,0.212700
1000,0.196500


TrainOutput(global_step=4375, training_loss=0.23446057521275113, metrics={'train_runtime': 897.7922, 'train_samples_per_second': 38.985, 'train_steps_per_second': 4.873, 'total_flos': 3205354521600000.0, 'train_loss': 0.23446057521275113, 'epoch': 5.0})

## **Save Model**

In [30]:
trainer.save_model(model_dir)
tokenizer.save_pretrained(model_dir)

('/content/drive/MyDrive/saved_t5_models/tokenizer_config.json',
 '/content/drive/MyDrive/saved_t5_models/special_tokens_map.json',
 '/content/drive/MyDrive/saved_t5_models/spiece.model',
 '/content/drive/MyDrive/saved_t5_models/added_tokens.json')

## **Load Model**

In [31]:
#load the model and tokenizer from the save directory
model = T5ForConditionalGeneration.from_pretrained(model_dir)
tokenizer = T5Tokenizer.from_pretrained(model_dir)

## **Pharaphrase Generation System**

In [32]:
import torch
# Set the device (if GPU avilable use it if not then use CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print (device)

cuda


In [35]:
model = model.to(device)


In [45]:
def preprocess_input(input_text):
    return "paraphrase: " + input_text


def generate_pharaphrase(input_sentence, model, tokenizer, max_length=75,
                         num_return_sequences=4, num_beams=5,
                         top_k=100, top_p=0.9, temperature=1.0):

    # Prepare input text
    input_text = preprocess_input(input_sentence)

    # Tokenize
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True,
                       max_length=max_length, padding='max_length')

    # Move tensors to device (GPU/CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Generate paraphrases
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=max_length + 20,
        num_return_sequences=num_return_sequences,
        num_beams=num_beams,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        do_sample=True,
        early_stopping=True
    )

    # Decode outputs to text
    paraphrased_sentences = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    return paraphrased_sentences



In [42]:
input_sentence = "the quick brown fox jumps over the lazy dogs"
paraphrased_sentences = generate_pharaphrase(input_sentence, model, tokenizer, max_length=75, num_return_sequences=4)



In [43]:
for sent in paraphrased_sentences:
    print(sent)

The quick brown fox jumps over the lazy dogs .
The quick brown fox jumps over the lazy dogs
The quick brown fox leaps over the lazy dogs.
The quick brown fox jumps over the lazy dogs ...


In [40]:
input_sentence = "She Enjoy reading books on rainy day"
paraphrased_sentences = generate_pharaphrase(input_sentence, model, tokenizer, max_length=75, num_return_sequences=4)

for i, sent in enumerate(paraphrased_sentences, start=1):
    print(f"pharaphrase:{i}. {sent}")

pharaphrase:1. She Enjoys reading books on rainy day.
pharaphrase:2. She enjoys reading books on rainy day.
pharaphrase:3. She Enjoy reading books on rainy day.
pharaphrase:4. She Enjoys reading books on rainy day:


The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


1: The short brown fox jumps over the lazy dog.
2: The quick brown fox jumps over the lazy dog .
3: The quick brown fox jumps over the lazy dog .
4: The quick brown fox jumps over the lazy dog.
5: The quick brown fox jumps over the lazy dog.
